# Report : Project of Basics of Mobile Robotics 

Group members :

- Cirillo Thomas
- Muller Nathan
- Sahin Ali Fuat
- Vinet Pierre

## Table of Contents

- [Introduction](#introduction)
- [1. Image Processing](#image-processing)
- [2. Path Finding](#path-finding)
- [3. Global Navigation](#global-navigation)
- [4. Local Obstacle Avoidance](#local-obstacle-avoidance)
- [5. Kalman Filter](#kalman-filter)
- [Conclusion](#conclusion)


## Introduction

### The Environment 

Our environment consist of a white surface with Aruco markers on the different corners to be able to identify them using the webcam given to us. We first decided to use a white surface with a black grid but then decided to go with a fully white surface instead because we use a binary threshold to detect our obstacles which would then interpret the grid as an obstacle. To correct this we would had needed to specifically extract the grid to create a mask out of it and substract it to our image in order to not detect it as an obstacle but we would not need this if we used a simple white surface. We also decided that the obstacle were not just squared one and therefor the use of a grid made less sense. 

The start and the goal are represented as Aruco markers as well. The obstacle used for the global navigation are black objects because as said before we use a binary threshold to detect them and blakc works the best for this situation, and the one used for the local avoidance are white objects because since they have the same color as the surface they are not detected with our image processing program.

### Object-Oriented Programming

We chose an object-oriented approach because it offers a structured and intuitive way to encapsulate data and functions into objects, providing a clear and organized framework for our project. This choice allows us to efficiently manage relationships between various components, and easily modify or add new functionalities. 

### Code Structure

The code is structured into different parts :

- Image Processing
- Kalman Filter
- Pathfinding
- Global Navigation
- Local Avoidance

There is a different .py file for every part, 

## 1. Image Processing

The goal of this part is to detect the different Aruco Markers at the corners of the environment in order to warp the image to focus on the wanted part of it. Then, the start and goal points are detected (they are represented as Aruco Markers) as well as the robot.

At first, the image is taken using the given webcam :

## 3. Path Planning

The goal of this section is to find the shortest path between two points in a grid-like structure while minimizing the total cost associated with the path. <br/>
We chose to use the A* star Algorithm because it it garanteed to find the shortest path and it is more efficient than other algorithms in terms of both time and memory usage.<br/>
We use this algorithm to find the shortest path from the robot initial position to the robot targeted position. At this point of the project we are taking into account only the global obstacles visible from the camera, and not the local ones.